<a href="https://colab.research.google.com/github/fatma18F/partnet/blob/main/PointNetSeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [3]:
root_dir = "/content/gdrive/My Drive"

!pip install path.py;
from path import Path
import sys
sys.path.append(root_dir)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import plotly.graph_objects as go
import numpy as np
import scipy.spatial.distance
import math
import random
import h5py
import json
import os
#import utils
from torchvision import transforms, utils

from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.utils.data.dataset import random_split

## partnet

In [5]:
def load_json(fn):
    with open(fn, 'r') as fin:
        return json.load(fin)

def load_h5(fn):
    with h5py.File(fn, 'r') as fin:
        pts = fin['pts'][:]
        gt_label = fin['gt_label'][:]
        gt_mask = fin['gt_mask'][:]
        gt_valid = fin['gt_valid'][:]
        gt_other_mask = fin['gt_other_mask'][:]
        return pts, gt_label, gt_mask, gt_valid, gt_other_mask

def load_data(fn):
    cur_json_fn = fn.replace('.h5', '.json')
    record = load_json(cur_json_fn)
    pts, gt_label, gt_mask, gt_valid, gt_other_mask = load_h5(fn)
    return pts, gt_label, gt_mask, gt_valid, gt_other_mask, record

In [6]:
path='/content/gdrive/MyDrive/ins_seg_h5_for_detection/Chair-1/train-00.h5'
cur_h5_fn = os.path.join(path)
print('Reading data from ', cur_h5_fn)
pts, gt_label, gt_mask, gt_valid, gt_other_mask, _ = load_data(cur_h5_fn)

Reading data from  /content/gdrive/MyDrive/ins_seg_h5_for_detection/Chair-1/train-00.h5


In [7]:
x=gt_mask[3][0]
np.count_nonzero(x == True)


3213

In [8]:
category='Chair'
level_id=1
data_in_dir = f'{root_dir}/ins_seg_h5_for_detection/%s-%d/' % (category, level_id)
train_h5_fn_list = []
for item in os.listdir(data_in_dir):
    if item.endswith('.h5') and item.startswith('train-'):
        train_h5_fn_list.append(item)
val_h5_fn_list = []
for item in os.listdir(data_in_dir):
    if item.endswith('.h5') and item.startswith('val-'):
        val_h5_fn_list.append(item)
print('train List: ', train_h5_fn_list)
print('val List: ', val_h5_fn_list)


train List:  ['train-03.h5', 'train-01.h5', 'train-04.h5', 'train-00.h5', 'train-02.h5']
val List:  ['val-00.h5']


In [9]:
pts_total=[]
for item in train_h5_fn_list:
         path='/content/gdrive/MyDrive/ins_seg_h5_for_detection/Chair-1/train-00.h5'
         cur_h5_fn = os.path.join(path)
         pts, gt_label, gt_mask, gt_valid, gt_other_mask, _ = load_data(cur_h5_fn)
         i=0
         pts_total=pts
         gt_label_total=gt_label
         for item in train_h5_fn_list:
            if i>0 : 
                cur_h5_fn = os.path.join(data_in_dir, item)
         
                pts, gt_label, gt_mask, gt_valid, gt_other_mask, _ = load_data(cur_h5_fn)
                #print("shape of pts" ,i, ":" ,pts.shape)
                pts_total=np.concatenate((pts_total, pts))
                gt_label_total=np.concatenate((gt_label_total, gt_label))
            i+=1
print("shape of pts: " ,pts_total.shape)
print("shape of gt_label: " ,gt_label_total.shape)

shape of pts:  (4489, 10000, 3)
shape of gt_label:  (4489, 10000)


In [10]:
'''path='/kaggle/input/partnet/partnet_seg_exps-master/exps/utils/provider.py'
provider_FILE = os.path.join(path)
os.system('cp %s %s' % ( provider_FILE, '/kaggle/working/')) # bkp of model def
import provider'''

"path='/kaggle/input/partnet/partnet_seg_exps-master/exps/utils/provider.py'\nprovider_FILE = os.path.join(path)\nos.system('cp %s %s' % ( provider_FILE, '/kaggle/working/')) # bkp of model def\nimport provider"

In [11]:
path='gdrive/MyDrive/ins_seg_h5_for_detection/provider.py'
provider_FILE = os.path.join(path)
os.system('cp %s %s' % ( provider_FILE, '.')) # bkp of model def
import provider

In [12]:
# shuffle data order
n_shape = pts.shape[0]
idx = np.arange(n_shape)
np.random.shuffle(idx)

pts = pts[idx, ...]
gt_label = gt_label[idx, ...]
gt_mask = gt_mask[idx, ...]
gt_valid = gt_valid[idx, ...]
gt_other_mask = gt_other_mask[idx, ...]

# data augmentation to pts
pts = provider.jitter_point_cloud(pts)
pts = provider.shift_point_cloud(pts)
pts = provider.random_scale_point_cloud(pts)
pts = provider.rotate_perturbation_point_cloud(pts)

In [13]:
class Data(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, pts,gt_label,gt_mask,gt_valid, transform=None):
        
        self.pts = pts
        self.gt_label = gt_label
        self.gt_mask = gt_mask
        self.gt_valid=gt_valid
       

    def __len__(self):
        return pts.shape[0]

    def __getitem__(self, idx):
        #if not self.valid:
          #  theta = random.random()*360
         #   image2 = utils.RandRotation_z()(utils.RandomNoise()(image2))
        
        return {'image': np.array(pts[idx], dtype="float32"), 'category': gt_label[idx].astype(int) , 'masks':gt_mask[idx], 'valid':gt_valid}

In [14]:
dset = Data(pts , gt_label, gt_mask, gt_valid)
train_num = int(len(dset) * 0.95)
val_num = int(len(dset) *0.05)
if int(len(dset)) - train_num -  val_num >0 :
    train_num = train_num + 1
elif int(len(dset)) - train_num -  val_num < 0:
    train_num = train_num -1
#train_dataset, val_dataset = random_split(dset, [3000, 118])
train_dataset, val_dataset = random_split(dset, [train_num, val_num])
val_dataset.valid=True

print('######### Dataset class created #########')
print('Number of images: ', len(dset))
print('Sample image shape: ', dset[0]['image'].shape)
#print('Sample image points categories', dset[0]['category'], end='\n\n')

train_loader = DataLoader(dataset=train_dataset, batch_size=8)
val_loader = DataLoader(dataset=val_dataset, batch_size=8)

#dataloader = torch.utils.data.DataLoader(dset, batch_size=4, shuffle=True, num_workers=4)

######### Dataset class created #########
Number of images:  1024
Sample image shape:  (10000, 3)


In [15]:
stat_in_fn = '/content/gdrive/MyDrive/ins_seg_h5_for_detection/Chair-1/chair-level1.txt'
print('Reading from ', stat_in_fn)
with open(stat_in_fn, 'r') as fin:
    part_name_list = [item.rstrip().split()[1] for item in fin.readlines()]
print('Part Name List: ', part_name_list)
NUM_CLASSES = len(part_name_list)
print('Semantic Labels: ', NUM_CLASSES)
NUM_INS = 200
print('Number of Instances: ', NUM_INS)

Reading from  /content/gdrive/MyDrive/ins_seg_h5_for_detection/Chair-1/chair-level1.txt
Part Name List:  ['chair/chair_head', 'chair/chair_back', 'chair/chair_arm', 'chair/chair_base', 'chair/chair_seat']
Semantic Labels:  5
Number of Instances:  200


## Model

In [16]:
n=np.unique(gt_label.ravel()).size
n

6

In [17]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=128)
        self.fc1 = nn.Conv1d(3,64,1)
        self.fc2 = nn.Conv1d(64,128,1) 
        self.fc3 = nn.Conv1d(128,128,1)
        self.fc4 = nn.Conv1d(128,512,1)
        self.fc5 = nn.Conv1d(512,2048,1)

        
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(2048)

   def forward(self, input):
        n_pts = input.size()[2]
        matrix3x3 = self.input_transform(input)
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)
        outs = []
        
        out1 = F.relu(self.bn1(self.fc1(xb)))
        outs.append(out1)
        out2 = F.relu(self.bn2(self.fc2(out1)))
        outs.append(out2)
        out3 = F.relu(self.bn3(self.fc3(out2)))
        outs.append(out3)
        matrix128x128 = self.feature_transform(out3)
        
        out4 = torch.bmm(torch.transpose(out3,1,2), matrix128x128).transpose(1,2) 
        outs.append(out4)
        out5 = F.relu(self.bn4(self.fc4(out4)))
        outs.append(out5)
       
        xb = self.bn5(self.fc5(out5))
        
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        out6 = nn.Flatten(1)(xb).repeat(n_pts,1,1).transpose(0,2).transpose(0,1)#.repeat(1, 1, n_pts)
        outs.append(out6)
        
        
        return outs, matrix3x3, matrix128x128


class PointNetSeg(nn.Module):
    def __init__(self, classes = 6):
        super().__init__()
        self.transform = Transform()

        self.fc1 = nn.Conv1d(3008,256,1) 
        self.fc2 = nn.Conv1d(256,256,1) 
        self.fc3 = nn.Conv1d(256,128,1) 
        self.fc4 = nn.Conv1d(128,6,1) 
        self.fc4_ins = nn.Conv1d(128,200,1) 
        

        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.bn3 = nn.BatchNorm1d(128)
        self.bn4 = nn.BatchNorm1d(6)
        self.bn4_ins = nn.BatchNorm1d(200)
        
        #self.logsoftmax = nn.LogSoftmax(dim=1)
        

    def forward(self, input):
        inputs, matrix3x3, matrix128x128 = self.transform(input)
        stack = torch.cat(inputs,1)
        
        xb = F.relu(self.bn1(self.fc1(stack)))
       
        xb = F.relu(self.bn2(self.fc2(xb)))
    
        xb = F.relu(self.bn3(self.fc3(xb)))
        
        output = F.relu(self.bn4(self.fc4(xb)))
        masks_output = F.relu(self.bn4_ins(self.fc4_ins(xb)))

        end_points = {}

        #return self.logsoftmax(output), matrix3x3, matrix128x128
        return output,masks_output,end_points, matrix3x3, matrix128x128


## Loss

In [18]:
#from partnet github
#def get_seg_loss(seg_pred, seg_gt, end_points):
 #   per_point_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=seg_pred, labels=seg_gt)
#in pytorch :
#import torch.nn.functional as F
#loss = F.nll_loss(F.softmax(input), target)
'''
m = nn.LogSoftmax(dim=1)
criterion2 = nn.NLLLoss()
x = torch.randn(1, 5)
y = torch.empty(1, dtype=torch.long).random_(5)
loss2 = criterion2(m(x), y)
print(loss2)'''

'\nm = nn.LogSoftmax(dim=1)\ncriterion2 = nn.NLLLoss()\nx = torch.randn(1, 5)\ny = torch.empty(1, dtype=torch.long).random_(5)\nloss2 = criterion2(m(x), y)\nprint(loss2)'

 A regularization loss (with weight 0.001) is added to the softmax  loss

In [19]:
def get_seg_loss (outputs, labels, m3x3, m128x128, end_points, alpha = 0.0001): 
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    #id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    #id128x128 = torch.eye(128, requires_grad=True).repeat(bs,1,1)
    #if outputs.is_cuda:
     #   id3x3=id3x3.cuda()
     #   id128x128=id128x128.cuda()
    #diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    #diff128x128 = id128x128-torch.bmm(m128x128,m128x128.transpose(1,2))
    logsoftmax = nn.LogSoftmax(dim=1)
    per_point_loss= criterion(logsoftmax(outputs), labels) #+ alpha * (torch.norm(diff3x3)+torch.norm(diff128x128)) / float(bs)
    end_points['per_point_seg_loss'] = per_point_loss
    per_shape_loss = torch.mean(per_point_loss, -1)
    end_points['per_shape_seg_loss'] = per_shape_loss
   # loss = torch.mean(per_shape_loss)
    return per_shape_loss, end_points 

In [20]:
def hungarian_matching(pred_x, gt_x, curnmasks):
    """ pred_x, gt_x: B x nmask x n_point
        curnmasks: B
        return matching_idx: B x nmask x 2 """
    batch_size = gt_x.shape[0]
    nmask = gt_x.shape[1]
    matching_score = np.matmul(gt_x, np.transpose(pred_x, axes=[0, 2, 1])) # B x nmask x nmask
    matching_score = 1 - np.divide(matching_score, np.maximum(np.expand_dims(np.sum(pred_x, 2), 1)+np.sum(gt_x, 2, keepdims=True) - matching_score, 1e-8))
    matching_idx = np.zeros((batch_size, nmask, 2)).astype('int32')
    curnmasks = curnmasks.astype('int32')
    for i, curnmask in enumerate(curnmasks):
        row_ind, col_ind = linear_sum_assignment(matching_score[i, :curnmask, :])
        matching_idx[i, :curnmask, 0] = row_ind
        matching_idx[i, :curnmask, 1] = col_ind
    return matching_idx

In [21]:
def iou(pred_x, gt_x, gt_conf, n_point, nmask, end_points):
    matching_idx = tf.py_func(hungarian_matching, [pred_x, gt_x, tf.reduce_sum(gt_conf,-1)], tf.int32) # B x nmask x 2
    matching_idx = tf.stop_gradient(matching_idx)
    end_points['matching_idx'] = matching_idx

    matching_idx_row = matching_idx[:, :, 0]
    idx = tf.where(tf.greater_equal(matching_idx_row, 0))
    matching_idx_row = tf.concat((tf.expand_dims(tf.cast(idx[:, 0], tf.int32), -1), tf.reshape(matching_idx_row, [-1, 1])), 1)
    gt_x_matched = tf.reshape(tf.gather_nd(gt_x, matching_idx_row), [-1, nmask, n_point])
    
    matching_idx_column = matching_idx[:, :, 1]
    idx = tf.where(tf.greater_equal(matching_idx_column, 0))
    matching_idx_column = tf.concat((tf.expand_dims(tf.cast(idx[:, 0], tf.int32), -1),tf.reshape(matching_idx_column, [-1, 1])), 1)
    pred_x_matched = tf.reshape(tf.gather_nd(pred_x, matching_idx_column), [-1, nmask, n_point])
    
    # compute meaniou
    matching_score = tf.reduce_sum(tf.multiply(gt_x_matched, pred_x_matched),2)
    iou_all = tf.divide(matching_score, tf.reduce_sum(gt_x_matched, 2) + tf.reduce_sum(pred_x_matched, 2) - matching_score + 1e-8)
    end_points['per_shape_all_iou'] = iou_all
    
    meaniou = tf.divide(tf.reduce_sum(tf.multiply(iou_all, gt_conf), 1), tf.reduce_sum(gt_conf, -1) + 1e-8) # B
    return meaniou, end_points

In [22]:
'''for i, data in enumerate(train_loader, 0):
            inputs,labels,gt_mask_pl,gt_valid_pl = data['image'].to(device),data['category'].to(device),data['masks'].to(device),data['valid'].to(device)
            outputs ,mask_pred,end_points, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
            break 

print(mask_pred.shape)
print(gt_mask_pl.shape)
print(gt_valid_pl.shape)
'''

"for i, data in enumerate(train_loader, 0):\n            inputs,labels,gt_mask_pl,gt_valid_pl = data['image'].to(device),data['category'].to(device),data['masks'].to(device),data['valid'].to(device)\n            outputs ,mask_pred,end_points, m3x3, m64x64 = pointnet(inputs.transpose(1,2))\n            break \n\nprint(mask_pred.shape)\nprint(gt_mask_pl.shape)\nprint(gt_valid_pl.shape)\n"

In [23]:
# B=8 K=200 N=10000
def get_ins_loss(mask_pred, mask_gt, gt_valid, end_points):
    """ Input:  mask_pred   B x K x N
                mask_gt     B x K x N
                gt_valid    B x K
    """
    num_ins = mask_pred.size()[1]
    num_point = mask_pred.size()[2]
    meaniou, end_points = iou(mask_pred, mask_gt, gt_valid, num_point, num_ins, end_points)
    end_points['per_shape_mean_iou'] = meaniou
    loss = - tf.reduce_mean(meaniou)
    return loss, end_points

## Training loop

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [25]:
pointnet = PointNetSeg()

In [26]:
pointnet.to(device);

In [27]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.001)

In [28]:
from tqdm import tqdm

def train(model, train_loader, val_loader=None,  epochs=10, save=True):
    for epoch in tqdm(range(epochs)): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs,labels,gt_mask_pl,gt_valid_pl = data['image'].to(device),data['category'].to(device),data['masks'].to(device),data['valid'].to(device)
            optimizer.zero_grad()
            outputs ,mask_pred,end_points, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
            loss ,end_points = get_seg_loss(outputs, labels, m3x3, m64x64,end_points)
            #ins_loss, end_points = ins_loss(mask_pred, gt_mask_pl, gt_valid_pl, end_points)

            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs,labels,gt_mask_pl,gt_valid_pl = data['image'].to(device),data['category'].to(device),data['masks'].to(device),data['valid'].to(device)
                    outputs ,mask_pred,end_points, m3x3, m64x64 = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0) * labels.size(1) ##
                    correct += (predicted == labels).sum().item()
            val_acc = 100 * correct / total
            print('Valid accuracy: %d %%' % val_acc)

        # save the model
        if save:
            torch.save(pointnet.state_dict(), root_dir+"/modelsSeg_partnet/total"+str(epoch)+"_"+str(val_acc))


In [24]:
#train(pointnet, train_loader, val_loader,  save=True)

  0%|          | 0/10 [00:00<?, ?it/s]

[1,    10] loss: 1.553
[1,    20] loss: 1.324
[1,    30] loss: 1.257
[1,    40] loss: 1.176
[1,    50] loss: 1.167
[1,    60] loss: 1.107
[1,    70] loss: 1.091
[1,    80] loss: 1.064
[1,    90] loss: 1.047
[1,   100] loss: 1.033
[1,   110] loss: 1.013
[1,   120] loss: 1.037


 10%|█         | 1/10 [01:24<12:42, 84.69s/it]

Valid accuracy: 70 %
[2,    10] loss: 1.004
[2,    20] loss: 1.021
[2,    30] loss: 0.970
[2,    40] loss: 0.961
[2,    50] loss: 0.958
[2,    60] loss: 0.927
[2,    70] loss: 0.923
[2,    80] loss: 0.900
[2,    90] loss: 0.890
[2,   100] loss: 0.869
[2,   110] loss: 0.867
[2,   120] loss: 0.903


 20%|██        | 2/10 [02:44<10:56, 82.05s/it]

...

Valid accuracy: 76 %
[9,    10] loss: 0.524
[9,    20] loss: 0.544
[9,    30] loss: 0.516
[9,    40] loss: 0.538
[9,    50] loss: 0.551
[9,    60] loss: 0.512
[9,    70] loss: 0.520
[9,    80] loss: 0.499
[9,    90] loss: 0.494
[9,   100] loss: 0.438
[9,   110] loss: 0.479
[9,   120] loss: 0.519


 90%|█████████ | 9/10 [12:07<01:20, 80.40s/it]

Valid accuracy: 76 %
[10,    10] loss: 0.492
[10,    20] loss: 0.510
[10,    30] loss: 0.492
[10,    40] loss: 0.501
[10,    50] loss: 0.516
[10,    60] loss: 0.484
[10,    70] loss: 0.495
[10,    80] loss: 0.471
[10,    90] loss: 0.469
[10,   100] loss: 0.413
[10,   110] loss: 0.454
[10,   120] loss: 0.499


100%|██████████| 10/10 [13:27<00:00, 80.78s/it]

Valid accuracy: 76 %


Pointnet  authors, in their implementation, did not use feature matrix regularization for semantic segmentation

## test : ** loss += feature_transform_regularizer**

In [37]:
pointnet = PointNetSeg()
#path='/content/gdrive/MyDrive/modelsSeg_partnet/newLoss14_83.58901960784314'
path='/content/gdrive/MyDrive/modelsSeg_partnet/total9_76.95529411764706'
pointnet.load_state_dict(torch.load(path))
#pointnet.load_state_dict(torch.load(root_dir+"/modelsSeg_partnet/14_83.21156862745099",map_location=torch.device('cpu')))

pointnet.eval()


PointNetSeg(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(128, 64, kernel_size

In [38]:
batch = next(iter(val_loader))
pred = pointnet(batch['image'].transpose(1,2))
pred_np = np.array(torch.argmax(pred[0],1));
acc = (pred_np==np.array(batch['category']))

resulting_acc = np.sum(acc, axis=1) / 10000
resulting_acc

array([0.7149, 0.885 , 0.9094, 0.9822, 0.8725, 0.8587, 0.7177, 0.8439])

In [39]:
x,y,z=np.array(batch['image'][1]).T
c = np.array(batch['category'][1]).T
#c = np.array(pred_np[0]).T

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, 
                                   mode='markers',
                                   marker=dict(
        size=30,
        color=c,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1.0
    ))])
fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [40]:
x,y,z=np.array(batch['image'][1]).T
c = pred_np[1].T

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, 
                                   mode='markers',
                                   marker=dict(
        size=30,
        color=c,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1
    ))])
fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

## test

In [29]:
pointnet = PointNetSeg()
path='/content/gdrive/MyDrive/modelsSeg_partnet/newLoss14_83.58901960784314'
pointnet.load_state_dict(torch.load(path))
#pointnet.load_state_dict(torch.load(root_dir+"/modelsSeg_partnet/14_83.21156862745099",map_location=torch.device('cpu')))

pointnet.eval()


PointNetSeg(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(128, 64, kernel_size

In [30]:
#pointnet = PointNetSeg()
#pointnet.load_state_dict(torch.load(root_dir+"/modelsSeg_partnet/14_83.21156862745099",map_location=torch.device('cpu')))
#pointnet.eval()

In [31]:
batch = next(iter(val_loader))
pred = pointnet(batch['image'].transpose(1,2))
pred_np = np.array(torch.argmax(pred[0],1));
pred_np

array([[5, 4, 2, ..., 4, 5, 3],
       [5, 2, 4, ..., 5, 5, 4],
       [4, 2, 4, ..., 2, 5, 2],
       ...,
       [5, 2, 4, ..., 5, 2, 2],
       [5, 4, 4, ..., 2, 4, 2],
       [5, 2, 4, ..., 5, 5, 2]])

In [32]:
pred_np.shape

(8, 10000)

In [33]:
acc = (pred_np==np.array(batch['category']))
acc

array([[ True,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [34]:
resulting_acc = np.sum(acc, axis=1) / 10000
resulting_acc

array([0.7941, 0.742 , 0.824 , 0.9218, 0.7264, 0.9421, 0.9338, 0.8532])

In [35]:
x,y,z=np.array(batch['image'][1]).T
c = np.array(batch['category'][1]).T
#c = np.array(pred_np[0]).T

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, 
                                   mode='markers',
                                   marker=dict(
        size=30,
        color=c,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1.0
    ))])
fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [36]:

x,y,z=np.array(batch['image'][1]).T
c = pred_np[1].T

fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, 
                                   mode='markers',
                                   marker=dict(
        size=30,
        color=c,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1
    ))])
fig.update_traces(marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()